In [166]:
#pip install transformers

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModel
import torchxrayvision as xrv
import time
from collections import Counter
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time
from torchsummary import summary
import os
from PIL import Image
from transformers import AutoModelForSequenceClassification

In [168]:
#!git clone https://github.com/ieee8023/covid-chestxray-dataset

In [169]:
data = xrv.datasets.COVID19_Dataset(imgpath="covid-chestxray-dataset/images/",csvpath="covid-chestxray-dataset/metadata.csv")

In [170]:
data

{'Aspergillosis': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Aspiration': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Bacterial': {np.float32(0.0): 487, np.float32(1.0): 48},
 'COVID-19': {np.float32(0.0): 193, np.float32(1.0): 342},
 'Chlamydophila': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Fungal': {np.float32(0.0): 512, np.float32(1.0): 23},
 'H1N1': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Herpes ': {np.float32(0.0): 532, np.float32(1.0): 3},
 'Influenza': {np.float32(0.0): 531, np.float32(1.0): 4},
 'Klebsiella': {np.float32(0.0): 526, np.float32(1.0): 9},
 'Legionella': {np.float32(0.0): 526, np.float32(1.0): 9},
 'Lipoid': {np.float32(0.0): 527, np.float32(1.0): 8},
 'MERS-CoV': {np.float32(0.0): 527, np.float32(1.0): 8},
 'MRSA': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Mycoplasma': {np.float32(0.0): 530, np.float32(1.0): 5},
 'No Finding': {np.float32(0.0): 520, np.float32(1.0): 15},
 'Nocardia': {np.float32(0.0): 531, np.float32(1.0): 4},
 'Pneumocystis': 

COVID19_Dataset num_samples=535 views=['PA', 'AP'] data_aug=None

In [171]:
df = data.csv

In [172]:
labels = []
for i in range(len(data)):
    pd.Series(dict(zip(data.pathologies,data[i]["lab"])))
    labels.append(pd.Series(dict(zip(data.pathologies,data[i]["lab"]))))

labels = pd.DataFrame(labels)

In [173]:
labels = labels['COVID-19']

In [174]:
text_model = AutoModelForSequenceClassification.from_pretrained(
    "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12",
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [175]:
img_model = xrv.models.DenseNet(weights="densenet121-res224-all")

In [176]:
img_model.eval()

XRV-DenseNet121-densenet121-res224-all

In [177]:
image_transforms = {
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [178]:
def save_images_and_prepare_data(data, save_dir="covid_images_organized"):
    os.makedirs(save_dir, exist_ok=True)

    image_paths = []
    texts = []
    labels = []

    for i in range(len(data)):
        #obtem imagem (normalizada entre 0-1)
        img = data[i]["img"]
        img = np.squeeze(img)

        #converte para uint8 e PIL
        img_uint8 = (img * 255).astype(np.uint8)
        pil_img = Image.fromarray(img_uint8)

        #obtem filename do DataFrame
        filename = data.csv.iloc[i]['filename']
        if not isinstance(filename, str):  #pula se for NaN ou inválido
            continue

        #caminho completo para salvar
        filepath = os.path.join(save_dir, filename)
        pil_img.save(filepath)
        image_paths.append(filepath)

        # Texto clínico associado, string ou vazio
        clinical_note = data.csv.iloc[i]['clinical_notes']
        texts.append(clinical_note if isinstance(clinical_note, str) else "")

        # Label binária para COVID-19
        lab_series = pd.Series(dict(zip(data.pathologies, data[i]["lab"])))
        labels.append(lab_series['COVID-19'])

    return image_paths, texts, labels


In [179]:
image_paths, texts, labels = save_images_and_prepare_data(data, save_dir="covid_images_organized")

In [180]:
#criando dataset multimodal, ou seja, que comparta as imagens e os textos
class MultimodalDataset(Dataset):
    def __init__(self, image_paths, texts, labels, tokenizer, transform=None):
        self.image_paths = image_paths
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        #imagem
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')  #abre imagem em greyscale
    
        if self.transform:
            image = self.transform(image)

        # Texto
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        label = torch.tensor(self.labels[idx]).long()

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'label': label
        }

In [181]:
dataset = MultimodalTextImageDataset(
    image_paths, texts, labels,
    tokenizer=tokenizer,
    image_transform=image_transforms['test'],
    max_len=128
)

loader = torch.utils.data.DataLoader(dataset, batch_size=335, shuffle=False)

In [182]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [183]:
index_interesse = img_model.pathologies.index("Pneumonia")  #não é covid, mas é só para experimentação gera;

In [184]:
for batch in loader:
    #inputs para os dois modelos
    ids = batch["input_ids"].to(device)
    mask = batch["attention_mask"].to(device)
    imgs = batch["image"].to(device)
    labels = batch["label"].to(device)

    #texto -> BlueBERT
    with torch.no_grad():
        output_text = text_model(input_ids=ids, attention_mask=mask)
        probs_text = torch.softmax(output_text.logits, dim=1)
        preds_text = torch.argmax(probs_text, dim=1)

    #imagem -> DenseNet
    with torch.no_grad():
        output_img = img_model(imgs)
        probs_img = torch.sigmoid(output_img[:, index_interesse])  # ex: pneumonia
        preds_img = (probs_img > 0.5).long()

    #comparar erros
    erro_texto = preds_text != labels
    erro_img = preds_img != labels
    erros_comuns = erro_texto & erro_img

In [185]:
preds_text_np = preds_text.cpu().numpy()
preds_img_np = preds_img.cpu().numpy()
labels_np = labels.cpu().numpy()

erro_texto = preds_text_np != labels_np
erro_img = preds_img_np != labels_np
erros_comuns = erro_texto & erro_img

print(f"Total de erros no texto: {erro_texto.sum()}")
print(f"Total de erros na imagem: {erro_img.sum()}")
print(f"Total de erros comuns: {erros_comuns.sum()}")

indices_erros_comuns = np.where(erros_comuns)[0]
print("Índices dos exemplos com erros em ambos (texto e imagem):", indices_erros_comuns)

Total de erros no texto: 92
Total de erros na imagem: 116
Total de erros comuns: 19
Índices dos exemplos com erros em ambos (texto e imagem): [ 69  70  71  72  73  98  99 113 118 119 135 136 137 138 145 164 176 178
 197]


In [ ]:
#pouco correlacionados! a intersecção é pequena!